In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
x_train, x_test, y_train, y_test = train_test_split(df[['age','affordibility']], df.bought_insurance, test_size = 0.2, random_state = 42)

In [5]:
x_train[:5]

,age,affordibility
17,58,1
22,40,1
11,28,1
13,29,0
15,55,1


In [6]:
x_test[:5]

,age,affordibility
9,61,1
25,54,1
8,62,1
21,26,0
0,22,1


In [7]:
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()

x_train_scaled['age'] = x_train_scaled['age'] / 100
x_test_scaled['age'] = x_test_scaled['age'] / 100

In [8]:
x_train_scaled[:5]

,age,affordibility
17,0.58,1
22,0.40,1
11,0.28,1
13,0.29,0
15,0.55,1


In [9]:
x_test_scaled[:5]

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1


In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones')
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(x_train_scaled, y_train, epochs = 5000)



Epoch 1/5000


1/1 [==============================] - 7s 7s/step - loss: 0.7428 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 43ms/step - loss: 0.7424 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 27ms/step - loss: 0.7420 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.7416 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 17ms/step - loss: 0.7411 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.7407 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.7403 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.7399 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 30ms/step - loss: 0.7395 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 21ms/step - loss: 0.7390 - accuracy: 0.5000
Epoch 11

In [11]:
model.evaluate(x_test_scaled, y_test)

1/1 [==============================] - 18s 18s/step - loss: 0.2647 - accuracy: 1.0000


[0.26470303535461426, 1.0]

In [12]:
model.predict(x_test_scaled)

1/1 [==============================] - 1s 1s/step


array([[0.81626713],
       [0.75767547],
       [0.8236742 ],
       [0.18819511],
       [0.38559136],
       [0.1959822 ]], dtype=float32)

In [13]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

In [16]:
coef, intercept = model.get_weights()

In [17]:
coef

array([[5.0183096],
       [1.1966311]], dtype=float32)

In [18]:
intercept

array([-2.7665412], dtype=float32)

## Manually Predict

In [20]:
x_train_scaled[:5]

,age,affordibility
17,0.58,1
22,0.40,1
11,0.28,1
13,0.29,0
15,0.55,1


In [30]:
import math
def sigmoid(y):
    return 1 / (1 + math.exp(-y))


def get_predict(age, affordibility):
    y = age * coef[0] + affordibility*coef[1] + intercept
    predicted = sigmoid(y)

    return predicted

In [23]:
x_test_scaled[:3]

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1


In [24]:
y_test[:3]

9     1
25    1
8     1
Name: bought_insurance, dtype: int64

In [31]:
get_predict(0.61, 1)

C:\Users\Himel\AppData\Local\Temp\ipykernel_13996\1765961113.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-y))


0.8162671453430502

In [32]:
get_predict(0.54, 1)

C:\Users\Himel\AppData\Local\Temp\ipykernel_13996\1765961113.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-y))


0.7576754559416229

## Gredient Descent

In [33]:
epsilon = 1e-15

def log_loss(y_true, y_predicted):
    y_predicted1 = [max(i,epsilon) for i in y_predicted]
    y_predicted2 = [min(i,(1-epsilon)) for i in y_predicted1]
    y_predicted3 = np.array(y_predicted2)


    return -np.mean(y_true*np.log(y_predicted3) + ((1-y_true)*np.log(1-y_predicted3)))

In [37]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def gradient_descent(age, affordibility, y_true, epochs, loss_thresold):
    w1=w2=0
    bias = 0
    n = len(age)
    learning_rate = 0.5


    for i in range(epochs):
        y = w1*age + w2*affordibility + bias
        y_predicted = sigmoid(y)
        loss = log_loss(y_true, y_predicted)
        
        d_w1 = np.mean(age*(y_predicted - y_true))
        d_w2 = np.mean(affordibility*(y_predicted - y_true))
        d_bias = np.mean((y_predicted - y_true))

        w1 = w1 - learning_rate * d_w1
        w2 = w2 - learning_rate * d_w2
        bias = bias - learning_rate * d_bias

        print(f'Epoch{i}-------w1: {w1},  w2: {w2}, bias: {bias}, loss: {loss}')

        if loss <= loss_thresold:
            break

    return w1,w2,bias
    

In [38]:
gradient_descent(x_train_scaled['age'],x_train_scaled['affordibility'],y_train, 1000, 0.4921)

Epoch0-------w1: 0.022386363636363624,  w2: 0.03409090909090909, bias: 0.0, loss: 0.6931471805599453
Epoch1-------w1: 0.043118857788549236,  w2: 0.06451374661806406, bias: -0.004005135860758641, loss: 0.6899825864397562
Epoch2-------w1: 0.06255668848163667,  w2: 0.09198110624229572, bias: -0.011118610800223245, loss: 0.6873440584215054
Epoch3-------w1: 0.08098286398677612,  w2: 0.11705630055410782, bias: -0.020633993183957255, loss: 0.6850465852814565
Epoch4-------w1: 0.09861997489786922,  w2: 0.14018411647404486, bias: -0.03199573898457511, loss: 0.6829747337687819
Epoch5-------w1: 0.11564294405550565,  w2: 0.16171576329138365, bias: -0.04476732987404629, loss: 0.6810563644665686
Epoch6-------w1: 0.13218917631355342,  w2: 0.18192877171972235, bias: -0.058605924624942025, loss: 0.6792462360010799
Epoch7-------w1: 0.14836657585844365,  w2: 0.20104272516794433, bias: -0.07324234271720936, loss: 0.6775158158370572
Epoch8-------w1: 0.1642598549012393,  w2: 0.21923164061597425, bias: -0.088

(5.023600307151037, 1.227941939834985, -2.8024475970302185)

## Create neural network class

In [45]:
def sigmoid(z):
        return 1 / (1 + np.exp(-z))

In [54]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0


    def fit(self, x, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(x['age'], x['affordibility'], y, epochs, loss_thresold)

    def predict(self, x_test):
        weighted_sum = self.w1 * x_test['age'] + self.w2 * x_test['affordibility'] + self.bias
        return sigmoid(weighted_sum)

    
    def gradient_descent(self, age, affordibility, y_true, epochs, loss_thresold):
        w1=w2=0
        bias = 0
        n = len(age)
        learning_rate = 0.5
    
    
        for i in range(epochs):
            y = w1*age + w2*affordibility + bias
            y_predicted = sigmoid(y)
            loss = log_loss(y_true, y_predicted)
            
            d_w1 = np.mean(age*(y_predicted - y_true))
            d_w2 = np.mean(affordibility*(y_predicted - y_true))
            d_bias = np.mean((y_predicted - y_true))
    
            w1 = w1 - learning_rate * d_w1
            w2 = w2 - learning_rate * d_w2
            bias = bias - learning_rate * d_bias

            if i%50 ==0:
                print(f'Epoch{i}-------w1: {w1},  w2: {w2}, bias: {bias}, loss: {loss}')
    
            if loss <= loss_thresold:
                print(f'Epoch{i}-------w1: {w1},  w2: {w2}, bias: {bias}, loss: {loss}')
                break
    
        return w1,w2,bias
         

In [58]:
custom_model = myNN()
custom_model.fit(x_train_scaled,y_train, epochs = 1000, loss_thresold = 0.4921)

Epoch0-------w1: 0.022386363636363624,  w2: 0.03409090909090909, bias: 0.0, loss: 0.6931471805599453
Epoch50-------w1: 0.8042984937582341,  w2: 0.6869907785261236, bias: -0.6899296371554089, loss: 0.6273087665944431
Epoch100-------w1: 1.5567442138818437,  w2: 0.9316868221095383, bias: -1.1830969159257276, loss: 0.5920954080212858
Epoch150-------w1: 2.264268198602743,  w2: 1.042969882046667, bias: -1.5536391012747022, loss: 0.5659338078414824
Epoch200-------w1: 2.9144134968784376,  w2: 1.1021805958898585, bias: -1.8600503968941189, loss: 0.5450417081998791
Epoch250-------w1: 3.5067402597684274,  w2: 1.1408980365852606, bias: -2.1271568625218036, loss: 0.5280283292443865
Epoch300-------w1: 4.045160935439026,  w2: 1.171592098129347, bias: -2.366491793850027, loss: 0.514052258114746
Epoch350-------w1: 4.534881089112611,  w2: 1.1991387156914162, bias: -2.5840540454981062, loss: 0.5024948240847995
Epoch400-------w1: 4.981168438788418,  w2: 1.2253632036429483, bias: -2.7834132166597274, loss:

In [59]:
custom_model.predict(x_test_scaled)

9     0.816062
25    0.757356
8     0.823483
21    0.182976
0     0.384779
12    0.190606
dtype: float64

In [61]:
model.predict(x_test_scaled)

1/1 [==============================] - 2s 2s/step


array([[0.81626713],
       [0.75767547],
       [0.8236742 ],
       [0.18819511],
       [0.38559136],
       [0.1959822 ]], dtype=float32)